## Day 38 Proyect: Exercise Tracker with GSheets

### Proyecto Día 38 – Exercise Tracker with Google Sheets 🏃📊

Este proyecto permite:
- Escribir en lenguaje natural tu actividad física diaria.
- Obtener automáticamente las calorías, duración e intensidad usando la API de Nutritionix.
- Registrar estos datos en una hoja de cálculo de Google Sheets con Sheety.

📦 Tecnologías utilizadas:
- `requests` y `datetime`
- API de [Nutritionix](https://www.nutritionix.com/business/api)
- API de [Sheety](https://sheety.co/)
- Autenticación con token (Bearer)
- Variables de entorno (`os.environ`)


#### Paso 1: Consultar Nutritionix con lenguaje natural 🧠

Usamos la API para interpretar frases como:  
➡️ "Corrí 5km y anduve en bici 30 minutos"

Nutritionix devuelve datos como duración, calorías y tipo de ejercicio.

In [1]:
import requests                     # La librería para trabajar con HTTP
from datetime import datetime       # La librería para trabajar con fechas
import os                           # La librería para trabajar con el sistema operativo
from dotenv import load_dotenv      # La librería para cargar variables de entorno

In [2]:
# Carga las variables del archivo .env
load_dotenv()  

True

In [3]:
# Datos de autenticación 
NUTRITIONIX_APP_ID = os.getenv("NUTRITIONIX_APP_ID")
NUTRITIONIX_API_KEY = os.getenv("NUTRITIONIX_API_KEY")

# Datos para la solicitud 
GENDER = "male"
WEIGHT_KG = 70
HEIGHT_CM = 175
AGE = 30

In [4]:
# URL del endpoint de Nutritionix para medición de ejercicio
exercise_endpoint = "https://trackapi.nutritionix.com/v2/natural/exercise"

# Headers para una petición autenticada
headers = {
    "x-app-id": NUTRITIONIX_APP_ID,
    "x-app-key": NUTRITIONIX_API_KEY,
    "Content-Type": "application/json"
}

# Solicitud de entrada al usuario
exercise_input = input("¿Qué ejercicio hiciste hoy? 🏋️‍♀️: ")

# Imprimir información del ejercicio
print(f"🏋️‍♀️ Ejericio realizado: {exercise_input}\n")

# Parámetros para la solicitud
params = {
    "query": exercise_input,
    "gender": GENDER,
    "weight_kg": WEIGHT_KG,
    "height_cm": HEIGHT_CM,
    "age": AGE
}

# Realizar la solicitud POST al endpoint de Nutritionix para obtener datos de ejercicio
nutritionix_response = requests.post(url=exercise_endpoint
                        ,json=params
                        ,headers=headers)

# Imprimir la respuesta de la API
print(nutritionix_response.json())


🏋️‍♀️ Ejericio realizado: 30 lagartijas, 40 sentadillas, 50 dumbell curls, 60 abdominales

{'exercises': [{'tag_id': 824, 'user_input': 'lagartijas', 'duration_min': 1.52, 'met': 3.8, 'nf_calories': 6.74, 'photo': {'highres': 'https://d2xdmhkmkbyw75.cloudfront.net/exercise/824_highres.jpg', 'thumb': 'https://d2xdmhkmkbyw75.cloudfront.net/exercise/824_thumb.jpg', 'is_user_uploaded': False}, 'compendium_code': 2022, 'name': 'push-up', 'description': None, 'benefits': None}, {'tag_id': 780, 'user_input': 'sentadillas', 'duration_min': 2.5, 'met': 5, 'nf_calories': 14.58, 'photo': {'highres': 'https://d2xdmhkmkbyw75.cloudfront.net/exercise/780_highres.jpg', 'thumb': 'https://d2xdmhkmkbyw75.cloudfront.net/exercise/780_thumb.jpg', 'is_user_uploaded': False}, 'compendium_code': 2052, 'name': 'squats', 'description': None, 'benefits': None}, {'tag_id': 766, 'user_input': 'curls', 'duration_min': 4, 'met': 3.5, 'nf_calories': 16.33, 'photo': {'highres': 'https://d2xdmhkmkbyw75.cloudfront.net/ex

In [5]:
# Datos de autenticación 
NUTRITIONIX_APP_ID = os.getenv("NUTRITIONIX_APP_ID")
NUTRITIONIX_API_KEY = os.getenv("NUTRITIONIX_API_KEY")

# Datos para la solicitud 
GENDER = "male"
WEIGHT_KG = 70
HEIGHT_CM = 175
AGE = 30

#### Paso 2: Enviar los datos a Google Sheets 🧾

Utilizamos la API de Sheety. Cada fila representa un ejercicio con:
- Fecha y hora
- Nombre del ejercicio
- Duración
- Calorías quemadas


In [7]:
# Datos de autenticación 
SHEET_ENDPOINT = os.getenv("SHEET_ENDPOINT")
SHEET_TOKEN = os.getenv("SHEET_TOKEN")

In [8]:
# Headers para una petición autenticada
headers_sheety = {
    "Authorization": f"Bearer {SHEET_TOKEN}"
}

# Obtener la fecha y hora actuales
today_date = datetime.now().strftime("%d/%m/%Y")
now_time = datetime.now().strftime("%H:%M:%S")

# Recorrer cada ejercicio y enviarlo a la hoja de cálculo
for exercise in nutritionix_response.json()["exercises"]:
    # Crear un diccionario con los parámetros para la solicitud
    sheet_input = {
        "workout": {
            "date": today_date,
            "time": now_time,
            "exercise": exercise["name"].title(),
            "duration": exercise["duration_min"],
            "calories": exercise["nf_calories"]
        }
    }

    # Realizar la solicitud POST al endpoint de Sheety para agregar datos a la hoja de cálculo
    sheet_response = requests.post(url=SHEET_ENDPOINT
                                  ,json=sheet_input
                                  ,headers=headers_sheety)

    # Imprimir la respuesta de la API de Sheety
    print(sheet_response.text)


{
  "workout": {
    "date": "08/05/2025",
    "time": "15:36:29",
    "exercise": "Push-Up",
    "duration": 1.52,
    "calories": 6.74,
    "id": 6
  }
}
{
  "workout": {
    "date": "08/05/2025",
    "time": "15:36:29",
    "exercise": "Squats",
    "duration": 2.5,
    "calories": 14.58,
    "id": 7
  }
}
{
  "workout": {
    "date": "08/05/2025",
    "time": "15:36:29",
    "exercise": "Weight Lifting",
    "duration": 4,
    "calories": 16.33,
    "id": 8
  }
}
{
  "workout": {
    "date": "08/05/2025",
    "time": "15:36:29",
    "exercise": "Sit-Ups",
    "duration": 10,
    "calories": 32.67,
    "id": 9
  }
}


![Exercise Tracker](./GSheetTracker_Production.png)

*GSheetTracker POST*

#### 🧠 Conclusión

Este proyecto es un excelente ejemplo de integración entre múltiples servicios:

- 🧠 Interpreta lenguaje natural con Nutritionix
- 📋 Registra información automáticamente en Google Sheets vía Sheety
- 🔐 Maneja autenticación con token Bearer
- 🔒 Utiliza variables de entorno (`os.environ`) para proteger claves API